# <center>__MÉTODOS NUMÉRICOS__</center>
## <center>__PROJETO DA UNIDADE 2__</center>
## <center>__TEMA: PageRank com Decomposições em Auto valores e Auto vetores__</center>
#### <center>__ALUNO: Arthur Mauricio Thomaz Soares__</center>

<div class="alert alert-block alert-info">
1. INTRODUÇÃO
</div>

O algoritmo usado no notebook de exemplo se baseia nos conceitos de SVD e Decomposição de Eigen, da disciplina de métodos numericos.
A partir desses métodos de decomposição de matrizes, o autor busca aplica-lós buscando extrair as correlações de páginas da Wikipédia e os seus respectivos links, calculando a "impotância" de um artigo a partir da sua correlação com outros artigos. Para isso, é usada uma matriz de adjacencia que interliga o artigo com seus respectivos links.
Estudando esse notebook e pesquisando sobre o assunto, é interessante ver a variedade de informações que podemos tirar de matrizes bem montadas e organizadas com os métodos de decomposição. Por exemplo, nesse [link]() é mostrado que podemos facilmente extrair a correlação de posts a partir de suas palavras apenas usando SVD

<div class="alert alert-block alert-info">
2. DESCRIÇÃO DO PROBLEMA
</div>

O problema apresentado no notebook é o "PageRank", que é básicamente a categorização de páginas web que é muito importante hoje em dia para empresas como o Google.
Com o PageRank, podemos encontrar de forma mais fácil e rápida páginas relacionadas a um assunto que buscamos, e páginas mais relevantes ao assunto desejado em menos tempo.
Para esse page rank, o programa acessa páginas da Wikipédia a partir de um arquivo com as informações dessas páginas já sumarizadas disponível na DBPédia.
A partir de métodos que serão explicados nos próximos tópicos, conseguimos classificar páginas por sua "importância" (calculada a partir das ligações dessa página com outras).

<div class="alert alert-block alert-info">
3. MÉTODOS APLICADOS À SOLUÇÃO
</div>

### SVD
A decomposição de valores singulares de uma matriz é uma maneira de diagonizar a sua matriz e esse método pode ser utilizado para **quaisquer** matrizes (sendo quadradas ou não).
Formulá:

- $M=U\Sigma V^*$

- U = Mátriz Unitária

- $\Sigma$ = matriz diagonal do tamanho de M com números reais não negativos.

$V^*$ = Vatriz unitária transposta conjugada de V
### Decomposição de Eigen
A decomposição de Eigen usa uma transformação linear para extrair um **eigenvector** da matriz, que também pode ser chamado de “característica do vetor”. Essa característica é calculada usando uma escalar $\lambda$, essa escalar é chamada **eigenvalue**.

$T(v) = \lambda v$

- T = transformação linear
- v = eigenvector de uma matriz A
- $\lambda$ = eigenvalue do vétor v


### Power Method
O Power Method é usado para encontrar um eigenvalue dominante e eigenvector para uma matriz n * n. Para isso, ele itera sobre um valor p para encontrar o $\lambda_1$ mais proximo do eigenvalue definitivo.
[Explicação do Power Method](https://www.youtube.com/watch?v=_PDyi5BVY-E)

<div class="alert alert-block alert-info">
4. IMPLEMENTAÇÃO
</div>

Inicialmente, o autor mostra algumas formas de medirmos processos do sistema operacional com Python, para facilitar o debug do nosso notebook. Para isso usamos as libs:
- tqdm
- sleep
- os
Além dessas bibliotecas também usamos o velho amigo numpy, scipy e o bz2 que serve para descomprimir os arquivos do DBPedia.

Para motivos de exemplificação, eu vou usar os arquivos de redirects e de links apenas da linguagem pt-BR, pois não consegui fazer o download dos arquivos completos da DBPédia.

In [124]:
PATH = 'data/dbpedia/'
filenames = ["redirects_br.nt", "page_links_br.nt"]

In [125]:
import os, numpy as np, pickle
from bz2 import BZ2File
from datetime import datetime
from pprint import pprint
from time import time
import tqdm.notebook as tqdm
from scipy import sparse

from urllib.request import urlopen

Fazemos o download de arquivos comprimidos do DBPedia, que contem os dados da Wikipedia normalizados para análise.
Os arquivos são grandes e o servidor não é muito rápido, então o download demora bastante.
Por isso vamos usar arquivos com os dados parciais.

In [126]:
PATH = 'data/dbpedia/'
URL_BASE = 'http://downloads.dbpedia.org/3.8/br/'
filenames = ["redirects_br.nt.bz2", "page_links_br.nt.bz2"]

for filename in filenames:
    if not os.path.exists(PATH+filename):
        print("Downloading '%s', please wait..." % filename)
        open(PATH+filename, 'wb').write(urlopen(URL_BASE+filename).read())

Com os arquivos baixados, vamos criar uma matriz de adjacencia com as paginas e seus respectivos links.
É importante salientar que as linhas do arquivo são do tipo:
```
<http://dbpedia.org/resource/AfghanistanHistory> <http://dbpedia.org/property/redirect>
<http://dbpedia.org/resource/History_of_Afghanistan>
```

In [127]:
redirects_filename = PATH + filenames[0]
page_links_filename = PATH + filenames[1]

DBPEDIA_RESOURCE_PREFIX_LEN = len("http://br.dbpedia.org/resource/")
SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)

# Removendo primeira e ultima linha por motivos de limpeza de dados
def get_lines(filename): return (line.split() for line in BZ2File(filename))


def get_redirect(targ, redirects):
    seen = set()
    while True:
        transitive_targ = targ
        targ = redirects.get(targ)
        if targ is None or targ in seen: break
        seen.add(targ)
    return transitive_targ


def get_redirects(redirects_filename):
    redirects = {}
    lines = get_lines(redirects_filename)
    res = {}
    #for src, _, targ, _ in tqdm.tqdm(lines):
    for arr in tqdm.tqdm(lines):
        key = arr[0][SLICE].decode("utf-8")
        res[key] = get_redirect(arr[2][SLICE].decode("utf-8"), redirects)
    return res


redirects = get_redirects(redirects_filename)
len(redirects)

0it [00:00, ?it/s]

14675

Com o dicionario criado, iremos criar um dicionario auxiliar para guardar o link e seu respectivo ID.

In [128]:
def add_item(lst, redirects, index_map, item):
    k = item[SLICE]
    redirect_target = redirects.get(k)
    res = True
    if redirect_target is None:
        res = False
        redirect_target = k
    update_index_map = index_map.setdefault(redirect_target, len(index_map))
    lst.append(update_index_map)
    return res

In [129]:
# Computing the integer index map
index_map = dict()  # links->IDs
lines = get_lines(page_links_filename)
source, destination, data = [], [], []
for l, split in tqdm.tqdm(enumerate(lines)):
    add_item(source, redirects, index_map, split[0].decode("utf-8"))
    add_item(destination, redirects, index_map, split[2].decode("utf-8"))
    data.append(1)

len(data)

0it [00:00, ?it/s]

1064862

Temos 309K de items no nosso índice!

Vamos fazer uns testes com nossos dados

In [130]:
index_map_aux = index_map.copy()

In [131]:
index_map_aux['Harrison_Ford']

149579

Os primeiros 10 artigos que citam Harrison Ford

In [132]:
[i for i, x in enumerate(source) if x == 149579][:10]

[845530,
 845531,
 845532,
 845533,
 845534,
 845535,
 845536,
 845537,
 845538,
 845539]

Esses acima são as paginas que citam Harrison Ford.
Vamos pegar um exemplo.

In [133]:
source[845577], destination[845577]

(149579, 251127)

In [134]:
for page_name, index in index_map.items():
    if index == 251127:
        print(page_name)

Golden_Globe_Award_for_Best_Actor_-_Motion_Picture_Drama


Indo no [link](https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Motion_Picture_Drama) da Wikipedia, podemos ver que realmente essa página cita o Harisson Ford.

<div class="alert alert-block alert-info">
5. CASOS DE USO
</div>

O caso de uso será utilizando o arquivo previamente carregado das páginas da Wikipedia traduzidas em PT-BR.

Vamos criar uma matriz esparsa com os dados carregados e normalizados.

In [135]:
n = len(data)
X = sparse.coo_matrix((data, (destination, source)), shape=(n, n), dtype=np.float32)
X = X.tocsr()

também salvamos os nomes das páginas em um dicionario separado para facilitar a analise.

In [136]:
names = {i: name for name, i in index_map.items()}

Para que não precisemos recalcular essa matriz esparsa, vamos salva-lá em um arquivo.
(O autor faz isso porque ele trabalha com muitos dados, mas vamos fazer também por motivos de aprendizado)

In [137]:
pickle.dump(X, open(PATH + 'X.pkl', 'wb'))
pickle.dump(index_map, open(PATH + 'index_map.pkl', 'wb'))

Para carregar os arquivos basta executar o seguinte código:

In [138]:
#X = pickle.load(open(PATH+'X.pkl', 'rb'))
#index_map = pickle.load(open(PATH+'index_map.pkl', 'rb'))
#names = {i: name for name, i in index_map.items()}

Agora, iremos usar o Power Method para extrair os tópicos mais relevantes da nossa matriz:

In [139]:
def show_ex(v):
    print(', '.join(names[i] for i in np.abs(v.squeeze()).argsort()[-1:-10:-1]))


def power_method(A, max_iter=100):
    n = A.shape[1]
    A_indices = A.indices
    A_sum = A.sum(axis=0).A1
    A.data /= np.take(A_sum, A_indices)

    scores = np.ones(n, dtype=np.float32) * np.sqrt(A.sum() / (n * n))  # initial guess
    for i in range(max_iter):
        scores = A @ scores
        nrm = np.linalg.norm(scores)
        scores /= nrm

    return scores

In [140]:
scores = power_method(X, 10)

In [141]:
show_ex(scores)

Rummad:Prederourien, Bro-C'hall, Saozneg, Breizh, Brezhoneg, Europa_(kevandir), Galleg, Rummad:Bloavezhio\u00F9, Yezh


Vamos tentar um resultado melhor com mais iterações:

In [142]:
scores = power_method(X)
show_ex(scores)

Rummad:Prederourien, Wikipedia:Kendivizo\u00F9_reolata/Anvio\u00F9_divoutin_gresianek_kozh_ha_modern, Degemer, Skoazell:FAG, Wikipedia:Politikerezh_stanka\u00F1, Wikipedia:Pajenno\u00F9_da_nulla\u00F1, Wikipedia:Lammit_dreist_ar_reolenno\u00F9, Patrom:Provi\u00F1s_Vercelli, Wikipedia:Diello\u00F9/Pajenno\u00F9_da_nulla\u00F1/rann_3_Ebrel_2006-Gwengolo_2006
